# Notebook di Training 
### Contesto: 
Una persona prende appuntamento dal medico, riceve tutte le istruzioni e non si presenta
### Obiettivo:
Costruire un classificatore che, dato un paziente e il suo appuntamento, prevede quando un paziente non si presenterà


In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder #per trasformare i valori object in valori numerici 
from sklearn.tree import DecisionTreeClassifier #per utilizzare come classificatore un DecisionTreeClassifier
from sklearn.metrics import accuracy_score #per valutare il modello 

### Dataset: 
è costituito da 14 colonne di cui la più importante è No-show, ossia se il paziente si presenterà o meno all'appuntamento.
Le features:
### PatientId: 
identificazione di un paziente 
### AppointmentId:
identificazione di un appuntamento 
### Gender:
maschio o femmina 
### ScheduledDay:
il giorno in cui qualcuno ha registrato l'appuntamento 
### AppointmentDay:
il giorno effettivo dell'appuntamento 
### Age:
quanti anni ha il paziente 
### Neighbourhood:
dove si svolge l'appuntamento
### Scholarship:
assicurazione familiare per la salute 
### Hipertension,Diabetes,Alcoholism, Handcap:
Si/No 
### SMS_received: 
1 o più messaggi invaiti al cliente 

### Preparazione dei dati 
Verifico se ci sono valori nulli nelle colonne e nelle righe, creo delle features aggiuntive che aiutano nelle classificazione, in particolare "WaitTime" che è il tempo trascorso tra il giorno della chiamata e il giorno dell'appuntamento e "weekday", il giorno corrispondente della settimana in cui si svolge l'appuntamento. Elimino le colonne che non sono rilevanti, come gli identificativi del paziente e dell'appuntamnento(PatientId, AppointmentId), il giorno della chiamata(ScheduledDay) e il luogo dove si svolge l'appuntamento(Neighbourhood)

In [ ]:
dataset_train = pd.read_csv('train.csv') #importo i dati dal csv che contiene i dati di train 

In [ ]:
dataset_test = pd.read_csv('test.csv') #importo i dati dal csv che contiene i dati di test

In [ ]:
dataset_train.head() #mostro le prime 5 righe del dataset di train 

In [ ]:
dataset_train.columns #mostro le colonne 

In [ ]:
#controllo che non ci siano righe nulle 
dataset_train.isna().any() #non ci sono righe nulle

In [ ]:
dataset_train = dataset_train.dropna() #elimino i valori nulli nelle colonne e nelle righe 

Decido di fare alcune modifiche alle colonne per ottenere feature più significative

In [ ]:
dataset_train.ScheduledDay = dataset_train.ScheduledDay.apply(np.datetime64) #trasformo in data
dataset_train.AppointmentDay = dataset_train.AppointmentDay.apply(np.datetime64) #trasformo in data 
dataset_train['WaitTime'] = (dataset_train.AppointmentDay - dataset_train.ScheduledDay).dt.days #creo una colonna che indica il tempo di attesa
dataset_train['weekday'] = dataset_train.AppointmentDay.dt.weekday #creo una colonna con il giorno della settimana

In [ ]:
#elimino le colonne che non mi sembrano rilevanti per l'addestramento
del[dataset_train['PatientId']]
del[dataset_train['AppointmentID']]
del[dataset_train['ScheduledDay']]
del[dataset_train['Neighbourhood']]

In [ ]:
dataset_train.columns #mostro le colonne rimaste

In [ ]:
dataset_train.head() 

In [ ]:
dataset_test.head() #mostro le prime 5 righe del dataset di test

In [ ]:
dataset_test.columns #mostro le colonne 

In [ ]:
#controllo che non ci siano righe nulle 
dataset_test.isna().any() #non ci sono righe nulle

In [ ]:
dataset_test = dataset_test.dropna() #elimino i valori nulli nelle colonne e nelle righe 

Decido di fare alcune modifiche alle colonne per ottenere feature più significative

In [ ]:
dataset_test.ScheduledDay = dataset_test.ScheduledDay.apply(np.datetime64) #trasformo in data
dataset_test.AppointmentDay = dataset_test.AppointmentDay.apply(np.datetime64) #trasformo in data 
dataset_test['WaitTime'] = (dataset_test.AppointmentDay - dataset_test.ScheduledDay).dt.days #creo una colonna che indica il tempo di attesa
dataset_test['weekday'] = dataset_test.AppointmentDay.dt.weekday #creo una colonna con il giorno della settimana

In [ ]:
#elimino le colonne che non sono significative 
del[dataset_test['PatientId']]
del[dataset_test['AppointmentID']]
del[dataset_test['ScheduledDay']]
del[dataset_test['Neighbourhood']]

In [ ]:
dataset_test.columns #mostro le colonne rimaste

In [ ]:
#individuo quelle che sono le features categoriche, ossia che sono object
features_categoriche_train = dataset_train.select_dtypes(include = 'object') 
features_categoriche_test = dataset_test.select_dtypes(include = 'object')


In [ ]:
#individuo le features che sono numeri 
features_numeriche_train = dataset_train.select_dtypes(include = np.number)   
features_numeriche_test = dataset_test.select_dtypes(include = np.number)   

In [ ]:
#con il label enncoder trasformo le feature che sono object in numeri 
label_encoder = LabelEncoder()

for i in features_categoriche_train:
    features_categoriche_train[i] = label_encoder.fit_transform(features_categoriche_train[i])
    
features_categoriche_train

In [ ]:
#con il label enncoder trasformo le feature che sono object in numeri 
label_encoder = LabelEncoder()

for i in features_categoriche_test:
    features_categoriche_test[i] = label_encoder.fit_transform(features_categoriche_test[i])
    
features_categoriche_test

In [ ]:
#concateno di nuovo tutte le feature del dataset di train e di test
new_train = pd.concat([features_numeriche_train, features_categoriche_train], axis = 1)
new_test = pd.concat([features_numeriche_test, features_categoriche_test], axis = 1)

### Definizione delle feature X e della variabile target y


In [ ]:
#dichiaro quale feature è X e quale è y
y_train, y_test = new_train['No-show'], new_test['No-show'] #la colonna target No-show
X_train, X_test = new_train.drop(columns=['No-show']), new_test.drop(columns=['No-show']) #tutte le altre feature del paziente

In [ ]:
X_train

In [ ]:
y_train

### Addestramento del modello

In [ ]:
#decido di utlizzare un DecisionTreeClassifier con i parametri entropy, random_state=1 
tree = DecisionTreeClassifier(criterion='entropy',random_state=1)
tree.fit(X_train,y_train)

Decido di usare come criterio 'entropy', ossia quello relativo all'information gain, il guadagno di informazione.

In [ ]:
#mostro la predizione sui dati di train 
predictions_train = tree.predict(X_train)

In [ ]:
#mostro l'accuratezza sui dati di train 
print("Accuracy : "+str(accuracy_score(y_train,predictions_train)))

In [ ]:
#effettuo la predizione sui dati di test 
predictions = tree.predict(X_test)

### Valutazione del modello

In [ ]:
#mostro l'accuratezza del modello sui dati di test
print("Accuracy : "+str(accuracy_score(y_test,predictions)))

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, predictions))

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

Ho preferito usare il decision tree classifier con questi parametri per non sbilanciare troppo la classe yes rispetto alla classe no,anche ottenendo un'accuracy inferiore